# AI Fashion Assistant v2.4.5 - Multi-Modal RAG

**Day 4: Visual-Aware RAG Pipeline**

---

**Project:** AI Fashion Assistant (TÜBİTAK 2209-A)  
**Student:** Hatice Baydemir  
**Date:** January 9, 2026  
**Version:** 2.4.5

---

## Goal

Create visual-aware RAG pipeline:
1. Load v2.2 RAG pipeline
2. Integrate multimodal retrieval results
3. Enhance prompts with visual attributes
4. Generate responses with visual reasoning
5. Test on sample queries
6. Quality evaluation

---

## PART 1: Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ai_fashion_assistant_v2')

print('Drive mounted')
print(f'Working directory: {os.getcwd()}')

Mounted at /content/drive
Drive mounted
Working directory: /content/drive/MyDrive/ai_fashion_assistant_v2


In [2]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import time

print('Imports complete')

Imports complete


---

## PART 2: Load GROQ LLM

In [3]:
# Install GROQ
!pip install -q groq

import os
from groq import Groq

# Initialize GROQ client
groq_api_key = os.getenv('GROQ_API_KEY')  # From Colab secrets
if not groq_api_key:
    groq_api_key = "GROQ_API_KEY"  # Or paste directly

client = Groq(api_key=groq_api_key)

print('GROQ LLM initialized')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 1.3 MB/s eta 0:00:00
GROQ LLM initialized


---

## PART 3: Load Day 3 Results

In [4]:
# Load product metadata
products_df = pd.read_csv('data/processed/meta_ssot.csv')

print(f'Loaded {len(products_df)} products')
print(f'Columns: {list(products_df.columns)}')

Loaded 44417 products
Columns: ['id', 'productDisplayName', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'gender', 'season', 'year', 'usage', 'desc', 'image_path', 'text_embedding', 'image_embedding', 'hybrid_embedding']


In [5]:
# Load Day 3 retrieval results
with open('v2.4.5-multimodal-rag/evaluation/results/retrieval_comparison.json', 'r') as f:
    retrieval_results = json.load(f)

print(f'Loaded retrieval results for {len(retrieval_results)} queries')
print(f'\nSample result keys: {list(retrieval_results[0].keys())}')

Loaded retrieval results for 5 queries

Sample result keys: ['product_id', 'product_name', 'query', 'text_only', 'image_only', 'multimodal', 'filtered', 'filter_scores']


In [6]:
# Load V2.1 visual attributes
v21_attrs_long = pd.read_csv('v2.1-core-ml-plus/evaluation/results/product_attributes.csv')

# Pivot to wide format
attr_df = v21_attrs_long.pivot_table(
    index='product_id',
    columns='category',
    values='value',
    aggfunc='first'
).reset_index()

print(f'Loaded attributes for {len(attr_df)} products')
print(f'Attribute categories: {[c for c in attr_df.columns if c != "product_id"][:5]}...')

Loaded attributes for 42388 products
Attribute categories: ['fit', 'formality', 'length', 'material_appearance', 'neckline']...


---

## PART 4: Visual-Aware RAG Pipeline Class

In [7]:
class VisualRAGPipeline:
    """RAG pipeline with visual attribute awareness"""

    def __init__(self,
                 client: Groq,
                 products_df: pd.DataFrame,
                 attr_df: pd.DataFrame,
                 model: str = "llama-3.3-70b-versatile"):
        """
        Args:
            client: GROQ client
            products_df: Product metadata
            attr_df: Visual attributes from V2.1
            model: LLM model name
        """
        self.client = client
        self.products_df = products_df
        self.attr_df = attr_df
        self.model = model

    def get_product_details(self, product_id: int) -> Dict:
        """Get product metadata and attributes"""
        # Product metadata
        prod = self.products_df[self.products_df['id'] == product_id]

        if len(prod) == 0:
            return None

        prod = prod.iloc[0]

        # Visual attributes
        attrs = self.attr_df[self.attr_df['product_id'] == product_id]

        details = {
            'id': product_id,
            'name': prod['productDisplayName'],
            'category': prod['articleType'],
            'color': prod['baseColour'],
            'gender': prod['gender'],
            'season': prod['season'],
            'usage': prod['usage']
        }

        # Add V2.1 visual attributes if available
        if len(attrs) > 0:
            attrs = attrs.iloc[0]
            details['pattern'] = attrs.get('pattern', 'N/A')
            details['style'] = attrs.get('style', 'N/A')
            details['material'] = attrs.get('material_appearance', 'N/A')

        return details

    def build_context(self,
                     query: str,
                     retrieved_products: List[int],
                     query_type: str = "multimodal",
                     top_k: int = 5) -> str:
        """Build RAG context with visual attributes"""

        context_parts = []
        context_parts.append(f"User Query: {query}")
        context_parts.append(f"Query Type: {query_type}")
        context_parts.append("\nTop Matching Products:\n")

        for i, prod_id in enumerate(retrieved_products[:top_k], 1):
            details = self.get_product_details(prod_id)

            if details:
                product_desc = f"""{i}. {details['name']}
   - Category: {details['category']}
   - Color: {details['color']}
   - Gender: {details['gender']}
   - Pattern: {details.get('pattern', 'N/A')}
   - Style: {details.get('style', 'N/A')}
   - Material: {details.get('material', 'N/A')}
   - Season: {details['season']}
   - Usage: {details['usage']}"""

                context_parts.append(product_desc)

        return "\n".join(context_parts)

    def generate_response(self,
                         query: str,
                         retrieved_products: List[int],
                         query_type: str = "multimodal",
                         include_visual_reasoning: bool = True) -> Dict:
        """Generate RAG response with visual awareness"""

        # Build context
        context = self.build_context(query, retrieved_products, query_type, top_k=5)

        # Build prompt
        if include_visual_reasoning:
            system_prompt = """You are a fashion assistant with expertise in visual style analysis.
When recommending products, consider not just categories and colors, but also:
- Visual patterns (striped, solid, checkered, etc.)
- Style aesthetics (minimalist, casual, formal, etc.)
- Material appearance (cotton-like, denim, silk, etc.)
- How these visual elements work together

Provide personalized recommendations explaining WHY products match visually."""
        else:
            system_prompt = "You are a helpful fashion shopping assistant."

        user_prompt = f"""{context}

Based on the user's query and the matching products above, provide a helpful recommendation.
Explain why these products match the query, highlighting visual similarities when relevant.
Keep response concise (3-4 sentences)."""

        # Call LLM
        start_time = time.time()

        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.7,
            max_tokens=200
        )

        response_time = time.time() - start_time

        return {
            'query': query,
            'query_type': query_type,
            'response': response.choices[0].message.content,
            'retrieved_products': retrieved_products[:5],
            'response_time': response_time,
            'model': self.model
        }

print('VisualRAGPipeline class defined')

VisualRAGPipeline class defined


---

## PART 5: Initialize Pipeline

In [8]:
# Initialize Visual RAG Pipeline
rag_pipeline = VisualRAGPipeline(
    client=client,
    products_df=products_df,
    attr_df=attr_df,
    model="llama-3.3-70b-versatile"
)

print('✓ VisualRAGPipeline initialized')
print(f'  Model: {rag_pipeline.model}')
print(f'  Products: {len(products_df)}')
print(f'  Attributes available: {len(attr_df)}')

✓ VisualRAGPipeline initialized
  Model: llama-3.3-70b-versatile
  Products: 44417
  Attributes available: 42388


---

## PART 6: Test Visual-Aware RAG

In [9]:
# Test on retrieval results from Day 3
rag_results = []

for result in retrieval_results:
    query = result['query']
    multimodal_products = result['multimodal']

    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"Retrieved {len(multimodal_products)} products via multimodal search")

    # Generate RAG response
    rag_response = rag_pipeline.generate_response(
        query=query,
        retrieved_products=multimodal_products,
        query_type="multimodal",
        include_visual_reasoning=True
    )

    print(f"\nResponse ({rag_response['response_time']:.2f}s):")
    print(rag_response['response'])

    rag_results.append(rag_response)

print(f"\n{'='*60}")
print(f"Generated {len(rag_results)} RAG responses")


Query: white shirts
Retrieved 10 products via multimodal search

Response (0.73s):
The top matching products are a great fit for the query "white shirts" as they all feature a predominantly white color palette. Visually, these products share a similar solid pattern, with some featuring a slight texture or subtle sheen, adding depth to the overall minimalist aesthetic. The cotton-like material appearance of these shirts also contributes to their cohesive look, making them versatile and easy to style. Overall, these products match the query due to their unified white color and simple, classic design.

Query: blue shirts
Retrieved 10 products via multimodal search

Response (0.63s):
The top matching products for "blue shirts" are a great fit due to their dominant blue color palette. Visually, they share similarities in their solid or subtly patterned designs, which complement the calming tone of blue. The cotton-like material appearance and casual style aesthetics of these shirts also co

---

## PART 7: Compare with v2.2 Baseline

In [10]:
# Generate baseline responses (without visual attributes)
baseline_results = []

for result in retrieval_results[:3]:  # Test on first 3
    query = result['query']
    text_products = result['text_only']  # Use text-only retrieval

    # Generate response WITHOUT visual reasoning
    baseline_response = rag_pipeline.generate_response(
        query=query,
        retrieved_products=text_products,
        query_type="text-only",
        include_visual_reasoning=False
    )

    baseline_results.append(baseline_response)

print(f'Generated {len(baseline_results)} baseline responses')

Generated 3 baseline responses


In [11]:
# Compare responses side-by-side
comparison_df = pd.DataFrame([
    {
        'query': rag_results[i]['query'],
        'baseline_response': baseline_results[i]['response'][:100] + '...',
        'visual_aware_response': rag_results[i]['response'][:100] + '...',
        'baseline_time': baseline_results[i]['response_time'],
        'visual_aware_time': rag_results[i]['response_time']
    }
    for i in range(len(baseline_results))
])

print('Response Comparison')
print('='*80)
print(comparison_df.to_string(index=False))
print('='*80)

Response Comparison
        query                                                                                       baseline_response                                                                                   visual_aware_response  baseline_time  visual_aware_time
 white shirts The top matching products are all white shirts, matching the user's query exactly. These products fe... The top matching products are a great fit for the query "white shirts" as they all feature a predomi...       0.470888           0.726877
  blue shirts I've found a selection of blue shirts that match your query. These products feature various shades o... The top matching products for "blue shirts" are a great fit due to their dominant blue color palette...       0.570354           0.632961
purple shirts The top matching products for "purple shirts" are a great fit, featuring various shades of purple fr... The top matching products feature a range of purple shirts with varying shades and styles. The

---

## PART 8: Response Quality Analysis

In [12]:
# Analyze response characteristics
quality_metrics = {
    'total_responses': len(rag_results),
    'avg_response_time': np.mean([r['response_time'] for r in rag_results]),
    'avg_response_length': np.mean([len(r['response']) for r in rag_results]),
    'min_response_time': np.min([r['response_time'] for r in rag_results]),
    'max_response_time': np.max([r['response_time'] for r in rag_results])
}

print('Response Quality Metrics')
print('='*60)
for metric, value in quality_metrics.items():
    print(f'{metric}: {value:.3f}' if isinstance(value, float) else f'{metric}: {value}')
print('='*60)

Response Quality Metrics
total_responses: 5
avg_response_time: 0.642
avg_response_length: 495.600
min_response_time: 0.581
max_response_time: 0.727


In [13]:
# Check for visual attribute mentions
visual_keywords = ['pattern', 'style', 'material', 'striped', 'solid', 'checkered',
                   'casual', 'formal', 'minimalist', 'cotton', 'denim', 'visual']

visual_mentions = []

for result in rag_results:
    response_lower = result['response'].lower()
    mentions = [kw for kw in visual_keywords if kw in response_lower]

    visual_mentions.append({
        'query': result['query'],
        'visual_keywords_count': len(mentions),
        'visual_keywords': mentions
    })

visual_df = pd.DataFrame(visual_mentions)

print('Visual Attribute Mentions in Responses')
print('='*60)
print(visual_df.to_string(index=False))
print('\nAverage visual keywords per response:', visual_df['visual_keywords_count'].mean())

Visual Attribute Mentions in Responses
        query  visual_keywords_count                                                               visual_keywords
 white shirts                      7                 [pattern, style, material, solid, minimalist, cotton, visual]
  blue shirts                      7                     [pattern, style, material, solid, casual, cotton, visual]
purple shirts                      9 [pattern, style, material, solid, casual, formal, minimalist, cotton, visual]
purple shirts                      7                     [pattern, style, material, solid, casual, cotton, visual]
  black jeans                      8           [style, material, solid, casual, minimalist, cotton, denim, visual]

Average visual keywords per response: 7.6


---

## PART 9: Save Results

In [14]:
# Save RAG results
EVAL_DIR = Path('v2.4.5-multimodal-rag/evaluation/results')
EVAL_DIR.mkdir(parents=True, exist_ok=True)

# Save all RAG responses
with open(EVAL_DIR / 'visual_rag_responses.json', 'w') as f:
    json.dump(rag_results, f, indent=2, default=str)
print(f'Saved: {EVAL_DIR / "visual_rag_responses.json"}')

# Save baseline comparison
comparison_df.to_csv(EVAL_DIR / 'rag_comparison.csv', index=False)
print(f'Saved: {EVAL_DIR / "rag_comparison.csv"}')

# Save quality metrics
with open(EVAL_DIR / 'rag_quality_metrics.json', 'w') as f:
    json.dump(quality_metrics, f, indent=2)
print(f'Saved: {EVAL_DIR / "rag_quality_metrics.json"}')

# Save visual mentions analysis
visual_df.to_csv(EVAL_DIR / 'visual_mentions.csv', index=False)
print(f'Saved: {EVAL_DIR / "visual_mentions.csv"}')

print('\n✓ All results saved')

Saved: v2.4.5-multimodal-rag/evaluation/results/visual_rag_responses.json
Saved: v2.4.5-multimodal-rag/evaluation/results/rag_comparison.csv
Saved: v2.4.5-multimodal-rag/evaluation/results/rag_quality_metrics.json
Saved: v2.4.5-multimodal-rag/evaluation/results/visual_mentions.csv

✓ All results saved


---

## Summary

In [15]:
print('='*60)
print('DAY 4: VISUAL-AWARE RAG COMPLETE')
print('='*60)

print('\nCompleted:')
print(f'  ✓ VisualRAGPipeline implemented')
print(f'  ✓ Visual attributes integrated into prompts')
print(f'  ✓ {len(rag_results)} queries tested')
print(f'  ✓ Baseline comparison (v2.2 vs v2.4.5)')
print(f'  ✓ Quality metrics analyzed')

print('\nKey Findings:')
print(f'  - Avg response time: {quality_metrics["avg_response_time"]:.2f}s')
print(f'  - Avg response length: {quality_metrics["avg_response_length"]:.0f} chars')
print(f'  - Visual keywords per response: {visual_df["visual_keywords_count"].mean():.1f}')

print('\nOutput Files:')
print('  - visual_rag_responses.json')
print('  - rag_comparison.csv')
print('  - rag_quality_metrics.json')
print('  - visual_mentions.csv')

print('\nNext Steps (Day 5):')
print('  1. Comprehensive evaluation metrics')
print('  2. Compare all versions (v2.0, v2.2, v2.4, v2.4.5)')
print('  3. Statistical analysis')
print('  4. Visualizations (charts, tables)')
print('  5. Final performance report')

print('='*60)

DAY 4: VISUAL-AWARE RAG COMPLETE

Completed:
  ✓ VisualRAGPipeline implemented
  ✓ Visual attributes integrated into prompts
  ✓ 5 queries tested
  ✓ Baseline comparison (v2.2 vs v2.4.5)
  ✓ Quality metrics analyzed

Key Findings:
  - Avg response time: 0.64s
  - Avg response length: 496 chars
  - Visual keywords per response: 7.6

Output Files:
  - visual_rag_responses.json
  - rag_comparison.csv
  - rag_quality_metrics.json
  - visual_mentions.csv

Next Steps (Day 5):
  1. Comprehensive evaluation metrics
  2. Compare all versions (v2.0, v2.2, v2.4, v2.4.5)
  3. Statistical analysis
  4. Visualizations (charts, tables)
  5. Final performance report
